In [ ]:
# !pip install pyreadr
# !pip install plotly

# !conda install -c conda-forge nodejs -y
# !conda install -c conda-forge/label/gcc7 nodejs -y
# !conda install -c conda-forge/label/cf201901 nodejs -y
# !conda install -c conda-forge/label/cf202003 nodejs -y

# !jupyter labextension install jupyterlab-plotly
# !pip install scipy
# !pip install -U kaleido

In [ ]:
import pyreadr
import os
import pandas as pd
import numpy as np
import plotly as plt
import plotly.express as px
import scipy
import plotly.figure_factory as ff
import kaleido

In [ ]:
ag_data = pd.read_csv("agdata_smoke.csv", sep=",", low_memory=False)
ag_data.head()

In [ ]:
ag_data.columns

W - exposure to smoking variable

In [ ]:
# from preprocessing R code
# data[data$smoking_frequency == "Daily",]$W <- 0 # raucher
# data[data$smoking_frequency == "Never",]$W <- 1 # nie-raucher

In [ ]:
smokers = ag_data[ag_data["W"] == 0]
non_smokers = ag_data[ag_data["W"] == 1]


print("Number of smokers - {0}".format(len(smokers)))
print("Number of non-smokers - {0}".format(len(non_smokers)))

In [ ]:
df = ag_data.copy()

# df["W"] = df["W"].astype(int)
df["W_str"] = df["W"].map({1: "No", 0: 'Yes'})

In [ ]:
### Sex

fig = px.histogram(df, x="W_str", color="sex", barnorm='percent', text_auto=".2f",
                   width=800, height=400)

fig.update_layout(
    title_text='AG Project: Ratio between male and female (non)smokers', # title of plot
    xaxis_title_text='Smoking', # xaxis label
    yaxis_title_text='Percentage', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()

#create plots dir
if not os.path.exists("plots"):
    os.mkdir("plots")

fig.write_image("plots/AG_sex.png")

In [ ]:
### Age

fig = px.histogram(df, x="W_str", color="age_cat", barnorm='percent', text_auto=".2f",
                   width=800, height=400)

fig.update_layout(
    title_text='AG Project: Age ratio between (non)smokers', # title of plot
    xaxis_title_text='Smoking', # xaxis label
    yaxis_title_text='Percentage', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()
fig.write_image("plots/AG_age.png")

In [ ]:
### BMI

hist_data = [smokers["bmi_corrected"], non_smokers["bmi_corrected"]]

group_labels = ['Smokers', 'Non-smokers']

colors = ['slategray', 'magenta']

fig = ff.create_distplot(hist_data, group_labels, bin_size=2, show_rug=False,
                         histnorm="probability density", colors=colors)


fig.update_layout(
    title_text='AG Project: BMI probability density of (non)smokers', # title of plot
    xaxis_title_text="BMI (kg/m2)", # xaxis label
    yaxis_title_text='Probability density', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.01 # gap between bars of the same location coordinates
)
fig.show()
fig.write_image("plots/AG_bmi.png")

## Create pairs of samples

In [ ]:
df_match = ag_data.copy()
df_match.shape

In [ ]:
df_match["is_treated"] = df_match["W"].astype(bool)
df_match["pair_nb"] = np.nan

df_match.shape

In [ ]:
# Optional weights for each covariate when computing the distances
# WARNING: the order of the items in scaling needs to be the same as the order of the covariates (i.e. columns)
scaling =  np.ones((df_match.shape[1],), dtype=int)

scaling.shape

In [ ]:
sex_ix = df_match.columns.get_loc("sex")
age_cat_ix = df_match.columns.get_loc("age_cat")
bmi_corrected_ix = df_match.columns.get_loc("bmi_corrected")

In [ ]:
# set the thresholds for each covariate, default is Inf (i.e. no matching)
thresholds =  np.empty((df_match.shape[1], ))
thresholds[:] = np.nan

# set particular values
thresholds[sex_ix] = 0
thresholds[age_cat_ix] = 1
thresholds[bmi_corrected_ix] = 4

thresholds.shape

# for i in [sex_ix, age_cat_ix, bmi_corrected_ix]:
#     print(thresholds[i])

In [ ]:
relevant_fields = ["sex", "age_cat", "bmi_corrected", "is_treated"]

In [ ]:
treated_units = df_match[df_match["is_treated"] == True]
control_units = df_match[df_match["is_treated"] == False]

print("Number of treated units: {0}".format(treated_units.shape[0]))
print("Number of control units: {0}".format(control_units.shape[0]))

In [ ]:
# pairwise difference between real-univariate covariate of treated VS control group
def pairDist(treated=np.array, control=np.array):
        
    D = treated[:, None] - control
    
    return D


# pairwise absolute difference between real-univariate covariates of treated VS control group
def abs_pairDist(treated=np.array, control=np.array):
        
    D = np.abs(treated[:, None] - control)
    
    return D


# pairwise difference between factor-valued (i.e. bounded integer-valued) covariates 
# (e.g. day of the week, month, ...) of treated VS control group, assuming the facotr levels are cyclic
# and only the shortest difference modulo nb_levels matters.
def pairModuloDist(treated, control, nb_levels):
    
    treated_control = pairDist(treated.astype(int), control.astype(int)) % nb_levels
    control_treated = pairDist(control.astype(int), treated.astype(int)) % nb_levels
    
    pmin = np.minimum(treated_control, np.transpose(control_treated))
    
    return pmin



# pairwise difference between covariates of treated VS control group
# Inputs: treated/control are of covariate vectors (one entry per unit, for a given covariate)
# Outputs: pairwise difference matrix
def is_categorical(array_like):
    return array_like.dtype.name == 'category'

def pairdifference(treated, control):
    #check the type of a treated dataframe column
    if is_categorical(treated.values[0]):
        nb_levels = length(levels(treated[1]))
        
        return pairModuloDist(treated, control, nb_levels)
    
    else:
        return pairAbsDistCpp(treated,control)

In [ ]:
cat = pd.Categorical(['a', 'b', 'c'])
is_categorical(cat)

In [ ]:
a = np.array([1, 2, 3])
b = np.array([2, 4, 5])

pairModuloDist(a, b, 2)